# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
len(data)

4870

In [11]:
len(data[data.race=='b'])

2435

In [13]:
len(data[data.race=='w'])

2435

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. What test is appropriate for this problem? Does CLT apply?
<br>Since we're gonna have two sets of data (for black sounding and white sounding names), we can use <b>two-sample bootstrap test</b> and <b>chi square test</b>. CLT applies here since the data is large.

2. What are the null and alternate hypotheses?
<br>H0: pw - pb = 0 (race has no impact on call back)
<br>HA: pw - pb > 0 (race has impact on call back, higher probability for whites)

In [28]:
w = data[data.race=='w']
b = data[data.race=='b']

In [29]:
w['call'].sum()

235.0

In [30]:
w_prob = w['call'].sum()/len(w)

In [31]:
b['call'].sum()

157.0

In [32]:
b_prob = b['call'].sum()/len(b)

In [33]:
print('w:',w_prob)
print('b:',b_prob)

w: 0.09650924024640657
b: 0.06447638603696099


In [34]:
# Your solution to Q3 here

- Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [48]:
reps = 100000

shift_b = b.call - np.mean(b.call) + data['call'].mean()
shift_w = w.call - np.mean(w.call) + data['call'].mean()

bs_b = np.empty(reps) 
bs_w = np.empty(reps) 

for i in range(reps):
    bs_b[i] = np.mean(np.random.choice(shift_b, size=len(shift_b)))
    bs_w[i] = np.mean(np.random.choice(shift_w, size=len(shift_w)))

bs_rep = bs_w - bs_b
ci = np.percentile(bs_rep, [2.5, 97.5])
margin_of_error = (ci[1] - ci[0]) /2
p = np.sum(bs_rep >= (w.call.mean()-b.call.mean())) / reps
print('Margin of error: ', margin_of_error,
      '\n95% confidence interval: ', ci,
      '\np-value =', p)

Margin of error:  0.01540040597319603 
95% confidence interval:  [-0.01560576  0.01519505] 
p-value = 0.0


Since p-value = 0, we reject H0

In [59]:
contigency = pd.crosstab(data.race == 'b', data.call == 1)
contigency.columns = ['No_call', 'Called']
contigency.index = ['White', 'Black']

In [60]:
contigency

,No_call,Called
White,2200,235
Black,2278,157


In [66]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [67]:
stat, p, dof, expected = chi2_contingency(contigency)
print('dof=%d' % dof)
print(expected)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

dof=1
[[2239.  196.]
 [2239.  196.]]
probability=0.950, critical=3.841, stat=16.449
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

- Write a story describing the statistical significance in the context or the original problem.

In our data, shows that white sounding names has higher percentage (9.65%) on callback after applying a job than black sounding names (6.44%). We want to see if race associates with callback.
<br>We did two tests: <b>two-sample bootstrap test</b> and <b>chi square test</b>. Both results did not support the hypothesis that race have the same probabilities to receive a callback after applying a job.

- Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [70]:
data.shape

(4870, 65)

I think just because we did the analysis on race, does not mean that it is the most important factor in callback success, as we can see in the data we have more than 60 other attributes that may also associate with the callback. I think to really know what are factors for callback success, we should do the analysis including other factors that are also relevant to callback success.